In [13]:
import json
from collections import defaultdict
import pandas as pd

pd.set_option("display.max_rows", None)

In [2]:
with open("knowledge_graph.projects.json", encoding="utf-8") as f:
    kg = json.load(f)

In [3]:
len(kg)

28412

In [14]:
df = pd.DataFrame(kg)

In [ ]:
df.info()

In [118]:
fields = defaultdict(int)

for item in kg:
    for k in item.keys():
        fields[k] += 1

In [119]:
new = [k for k in kg if k.get("category") is not None and k.get("category").lower() in ["PFPs".lower(), "Art".lower(), "Dexes".lower(), "Gaming".lower(), "game"]]
dn = pd.DataFrame(new)

In [120]:
dn.notnull().sum().sort_values(ascending=False).head(None)

_id                           9574
sources                       9574
name                          9574
category                      9574
id                            9574
imgUrl                        9572
lastUpdated                   9348
lastUpdatedAt                 9348
description                   9259
deployedChains                9240
tokenAddresses                8691
volume                        8053
numberOfOwners                8053
numberOfItems                 8053
volumeChangeLogs              8046
volumeChangeRate              8033
numberOfListedItems           8025
idOpensea                     8025
links                         8014
marketShareNFT                7806
rankNFT                       7806
priceChangeLogs               7756
price                         7558
infoChecked                   2823
idDefiLlama                   1274
tvl                           1234
tvlChangeLogs                 1233
socialAccounts                1220
marketShareDefi     

In [121]:
chains = defaultdict(list)
cnt = 0
for chain_list, prj in dn[["deployedChains", "id"]].values:
    if isinstance(chain_list, (list, tuple)):
        for c in chain_list:
            chains[c].append(prj)
    else:
        cnt += 1
print("not found chain:", cnt)

not found chain: 334


In [122]:
chains.keys()

dict_keys(['0x1', '0x89', '0x38', '0xa4b1', '0xa86a', '0xfa', '0x2b6653dc', '0xa'])

In [124]:
import psycopg2

conn = psycopg2.connect(
    database="postgres",
    host="34.126.75.56",
    port="5432",
    user="student_token_transfer",
    password="svbk_2023"
)

In [125]:
cur = conn.cursor()

In [136]:
def test(chain_id: str):
    cur.execute(f"SELECT DISTINCT(project) FROM chain_{chain_id}.smart_contract")
    db_project = cur.fetchall()
    print("num project in postgres, chain", chain_id, "=", len(db_project))
    db_project = [d[0] for d in db_project]
    nf = []

    for c in chains[chain_id]:
        if c not in db_project:
            nf.append(c)
    print("not found:", len(nf))
    print("num project in mongodb, chain", chain_id, "=", len(chains[chain_id]))

In [137]:
for chain_id in chains:
    print(chain_id)
    test(chain_id)
    print()

0x1
num project in postgres, chain 0x1 = 7953
not found: 4333
num project in mongodb, chain 0x1 = 5397

0x89
num project in postgres, chain 0x89 = 665
not found: 2553
num project in mongodb, chain 0x89 = 2577

0x38
num project in postgres, chain 0x38 = 217
not found: 345
num project in mongodb, chain 0x38 = 364

0xa4b1
num project in postgres, chain 0xa4b1 = 550
not found: 319
num project in mongodb, chain 0xa4b1 = 344

0xa86a
num project in postgres, chain 0xa86a = 396
not found: 206
num project in mongodb, chain 0xa86a = 221

0xfa
num project in postgres, chain 0xfa = 114
not found: 63
num project in mongodb, chain 0xfa = 68

0x2b6653dc
num project in postgres, chain 0x2b6653dc = 0
not found: 11
num project in mongodb, chain 0x2b6653dc = 11

0xa
num project in postgres, chain 0xa = 270
not found: 105
num project in mongodb, chain 0xa = 119

